## import libraries

In [1]:
#import gensim for topic modelling
import gensim # https://radimrehurek.com/gensim/models/ldamodel.html
from gensim import corpora
from gensim.models import CoherenceModel #to calculate coherence
import time #to have today's date
timestr = time.strftime("%Y%m%d")
import pandas as pd

In [2]:
#get the clean text from the other notebook
%run ./p_03_prepare_text_paper.ipynb
# doc_clean = get_doc_clean() ##all interviews together
#dict_clean_doc_x ##with different lenghts dropped

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## drop short documents

In [3]:
var_drop_x = 10 #drop all documents with number of words <= x. Only keep documents >x
doc_clean_x = dict_clean_doc_x['>'+str(var_drop_x)]  #from p_03_prepare text
doc_list_paras_x = dict_doc_list_paras_x['>'+str(var_drop_x)] #from p_03_prepare text

## convert corpus into a matrix

A DT matrix describes the frequency of terms that occur in a collection of documents. In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms

### creating the term dictionary

In [4]:
# Creating the term dictionary of our corpus, where every unique term is assigned an index. 
def get_dictionary(doc_clean):
    dictionary = corpora.Dictionary(doc_clean)
    return(dictionary)

In [5]:
#create dictionaries
dictionary_x = get_dictionary(doc_clean_x)

### creating the document term matrix

In [8]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#this is also called Bag of Words corpus
def get_doc_term_matrix(dictionary, doc_clean):
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    return(doc_term_matrix)

In [9]:
#get document term matrix
doc_term_matrix_x = get_doc_term_matrix(dictionary_x, doc_clean_x)

## Running the LDA Model

create an object for LDA model and train it on Document-Term matrix

In [12]:
# Creating the object for LDA model using gensim library + # Running and Trainign LDA model on the document term matrix.
def get_ldamodel(var_alpha, var_no_topics, var_passes, doc_term_matrix, dictionary):
    ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=var_no_topics, alpha=var_alpha,
                id2word = dictionary, random_state=1, passes=var_passes,minimum_probability=0) 
    #use random_state for reproducity
    return(ldamodel)

In [13]:
#choose parameters, run model:
var_alpha = 0.01
var_no_topics = 22
var_passes = 50 # default: passes=1
ldamodel = get_ldamodel(var_alpha, var_no_topics, var_passes, doc_term_matrix_x, dictionary_x)

#compare model results with the ones from the notebook p_05_set_parameters_LDA <br>
coherence_model_lda = CoherenceModel(model=ldamodel, \
                texts=doc_clean_x, dictionary=dictionary_x, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence() #get the coherence score
print(coherence_lda)

In [14]:
#do list with the topics keywords for graphs
list_topics_keywords = list()
for x in range(0, var_no_topics):
    wp = ldamodel.show_topic(x, topn=20) #default: topn=10, number of words
    topic_keywords = ", ".join([word for word, prop in wp])
    list_topics_keywords.append(topic_keywords)

In [15]:
#make dataframe with original topic numbers and the keywords
df_ori_num_keyw = pd.DataFrame({'ori_topic_num':range(0, len(list_topics_keywords)), 'keywords':list_topics_keywords})
df_ori_num_keyw.to_csv('.//results//topics_group//topic_original_numbers_keywords_'+timestr+'_V01.csv',
    sep=';', decimal=',')

In [16]:
list_topics_keywords

['people, work, political, talk, time, community, government, right, take, interest, good, try, way, need, sort, different, happen, party, back, much',
 'everyones, metro, insurance, expenditure, charter, speaking, correct, stance, speculation, represent, participated, 47, satisfied, uso, resident, doom, annex, logically, batteries, contentious',
 'job, labor, worker, trade_union, secretary, employment, just_transition, create, lose, sector, prime_minister, vice, cooperative, joint, people, renewables, president, information, transition, decommission',
 'mining, environmental, mine, sector, national, ministry, project, level, example, resource, minister, many, social, company, important, impact, law, area, government, activity',
 'water, river, momentum, crc, younger, aquifer, beis, monthly, sponsor, shell, crop, guajira, rain, dry, divert, thatll, pass, lately, lab, sierra',
 'state, india, government, power, electricity, pay, subsidy, give, indonesia, land, people, public, problem, r